In [12]:
import pandas as pd
import numpy as np

bee_data_to_load = "Resources/honey_production.csv"

bee_data = pd.read_csv(bee_data_to_load)
bee_data.head()


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Commodity,Data Item,Domain,Domain Category,Value
0,SURVEY,2021,MARKETING YEAR,NaN,STATE,ALABAMA,1.0,HONEY,"HONEY - PRICE RECEIVED, MEASURED IN $ / LB",TOTAL,NOT SPECIFIED,5.99
1,SURVEY,2021,MARKETING YEAR,NaN,STATE,ALABAMA,1.0,HONEY,"HONEY - PRODUCTION, MEASURED IN $",TOTAL,NOT SPECIFIED,"1,917,000"
2,SURVEY,2021,MARKETING YEAR,NaN,STATE,ALABAMA,1.0,HONEY,"HONEY - PRODUCTION, MEASURED IN LB",TOTAL,NOT SPECIFIED,"320,000"
3,SURVEY,2021,MARKETING YEAR,NaN,STATE,ALABAMA,1.0,HONEY,"HONEY - PRODUCTION, MEASURED IN LB / COLONY",TOTAL,NOT SPECIFIED,40
4,SURVEY,2021,MARKETING YEAR,NaN,STATE,ALABAMA,1.0,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"8,000"


In [13]:
#drop unecessary columns
bee_data = bee_data.drop(["Program", "Period", "Week Ending", "Geo Level", "State ANSI", "Commodity", "Domain", "Domain Category"], axis=1)

In [14]:
bee_data

,Year,State,Data Item,Value
0,2021,ALABAMA,"HONEY - PRICE RECEIVED, MEASURED IN $ / LB",5.99
1,2021,ALABAMA,"HONEY - PRODUCTION, MEASURED IN $","1,917,000"
2,2021,ALABAMA,"HONEY - PRODUCTION, MEASURED IN LB","320,000"
3,2021,ALABAMA,"HONEY - PRODUCTION, MEASURED IN LB / COLONY",40
4,2021,ALABAMA,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...","8,000"
...,...,...,...,...
4600,2000,WYOMING,"HONEY - PRICE RECEIVED, MEASURED IN CENTS / LB",59
4601,2000,WYOMING,"HONEY - PRODUCTION, MEASURED IN $","2,140,000"
4602,2000,WYOMING,"HONEY - PRODUCTION, MEASURED IN LB","3,627,000"
4603,2000,WYOMING,"HONEY - PRODUCTION, MEASURED IN LB / COLONY",93


In [15]:
honey_data = pd.pivot_table(bee_data, values = "Value", index=["Year", "State"], columns = "Data Item", aggfunc=np.sum)

In [16]:
honey_data = honey_data.reset_index()

In [17]:
#replsvr nan values with 0
honey_data = honey_data.fillna(0)

In [18]:
#remove commas from all values
honey_data = honey_data.replace(",","", regex=True)

#convert column values to float
honey_data["HONEY - PRICE RECEIVED, MEASURED IN $ / LB"] = honey_data["HONEY - PRICE RECEIVED, MEASURED IN $ / LB"].astype(float)
honey_data["HONEY - PRICE RECEIVED, MEASURED IN CENTS / LB"] = honey_data["HONEY - PRICE RECEIVED, MEASURED IN CENTS / LB"].astype(float)
honey_data["HONEY - PRODUCTION, MEASURED IN $"] = honey_data["HONEY - PRODUCTION, MEASURED IN $"].astype(int)
honey_data["HONEY - PRODUCTION, MEASURED IN LB"] = honey_data["HONEY - PRODUCTION, MEASURED IN LB"].astype(int)
honey_data["HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES"] = honey_data["HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES"].astype(int)
honey_data

Data Item,Year,State,"HONEY - PRICE RECEIVED, MEASURED IN $ / LB","HONEY - PRICE RECEIVED, MEASURED IN CENTS / LB","HONEY - PRODUCTION, MEASURED IN $","HONEY - PRODUCTION, MEASURED IN LB","HONEY - PRODUCTION, MEASURED IN LB / COLONY","HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES"
0,2000,ALABAMA,0.00,59.0,736000,1248000,78,16000
1,2000,ARIZONA,0.00,73.0,1723000,2360000,59,40000
2,2000,ARKANSAS,0.00,57.0,2757000,4836000,93,52000
3,2000,CALIFORNIA,0.00,59.0,18172000,30800000,70,440000
4,2000,COLORADO,0.00,62.0,1079000,1740000,60,29000
...,...,...,...,...,...,...,...,...
916,2021,VIRGINIA,8.23,0.0,1975000,240000,40,6000
917,2021,WASHINGTON,2.52,0.0,7741000,3072000,32,96000
918,2021,WEST VIRGINIA,4.80,0.0,1238000,258000,43,6000
919,2021,WISCONSIN,2.81,0.0,5547000,1974000,47,42000


In [19]:
#convert cents to dollars
cents_to_dollars = honey_data["HONEY - PRICE RECEIVED, MEASURED IN CENTS / LB"]*.01

#add them in new column
final_dollar_prices = honey_data["HONEY - PRICE RECEIVED, MEASURED IN $ / LB"]+cents_to_dollars

In [20]:
#create new dataframe
honey_production = pd.DataFrame({
    "Year": honey_data["Year"],
    "State": honey_data["State"],
    "Price Received, Measured in $/Lb": final_dollar_prices,
    "Production, Measured in Lb": honey_data["HONEY - PRODUCTION, MEASURED IN LB"],
    "Production, Measured in Lb/Colony": honey_data["HONEY - PRODUCTION, MEASURED IN LB / COLONY"],
    "Honey Bee Colonies": honey_data["HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES"]
})

In [21]:
# Exporting as CSV in the Resources Folder
honey_production.to_csv('Resources/honey_data_clean.csv')